<a href="https://colab.research.google.com/github/RuthKassahun/Skin-Lesion-Classification/blob/main/other_GLCM%2BColorHist%2BLBP_MultiClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
from google.colab import drive
import seaborn as sns
import pandas as pd
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics
from pathlib import Path
import xgboost as xgb
import lightgbm as lgb
from PIL import Image, ImageOps
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score, recall_score, confusion_matrix


In [ ]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##LBP

In [ ]:
def compute_lbp(arr):
  # LBP function params
  radius = 3
  n_points = 8 * radius
  n_bins = n_points + 2
  lbp = local_binary_pattern(arr,n_points,radius,'uniform')
  lbp = lbp.ravel()
  #feature_len = int(lbp.max() + 1)
  feature = np.zeros(n_bins)
  for i in lbp:
    feature[int(i)] += 1 # Feature
  feature /= np.linalg.norm(feature, ord=1) # Normalizing feature
  return feature

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


LBP train data

In [ ]:
SIZE = 224
train_images_lbp = []
train_labels_lbp = [] 
#for directory_path in glob.glob("cell_images/train/*"):
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_train/*"):
    label_lbp_train = directory_path.split("/")[-1]
    label = directory_path.split("/")[-1]
    #print(label_lbp_train)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img1 = cv2.imread(img_path,0) #Reading color images
        img1 = cv2.resize(img1, (SIZE, SIZE)) #Resize images
        arr = np.array(img1) # convert the image to numpy array
        feature_train_lbp = compute_lbp(arr)
        train_images_lbp.append(feature_train_lbp)
        train_labels_lbp.append(label_lbp_train)

train_images_lbp =np.array(train_images_lbp)
trait_labels_lbp =np.array(train_labels_lbp)


lbp val data

In [ ]:
t_image = []
test_image_id= []
t_lbp=[]
t_image_glcm=[]
t_hist=[]

for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):

        src2 = cv2.imread(os.path.join(img_path),1)
        kernel2 = cv2.getStructuringElement(1,(19,19))
        grayScale2 = cv2.cvtColor(src2, cv2.COLOR_BGR2GRAY )
        eqhist2=cv2.equalizeHist(grayScale2) #clahe
        blackhat2 = cv2.morphologyEx(eqhist2, cv2.MORPH_BLACKHAT, kernel2) #2 Perform the blackHat filtering on the gray
        ret,thresh3 = cv2.threshold(blackhat2,10,255,cv2.THRESH_BINARY) # intensify the hair countours in preparation for the inpainting algorithm
        dst2 = cv2.inpaint(src2,thresh3,1,cv2.INPAINT_TELEA)
        dst2=cv2.cvtColor(dst2,cv2.COLOR_RGB2GRAY)
        #f2=cv2.medianBlur(dst2,5)
        img2 = cv2.resize(dst2, (SIZE, SIZE)) #Resize images
        arr = np.array(img2) # convert the image to numpy array
        feature_test = compute_lbp(arr) #lbp
        t_images_lbp.append(feature_test)
        t_labels_lbp.append(label_lbp_test)
        #colorhist
        f2c=cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        img4=hist_features(f2c)
        t_hist.append(img4)
        #test_label_hist.append(labelhist_test)
        t_image_glcm.append(img2)

      
        t_image.append(img2)
        #t_lbp.append(feature_test)
        #call the function
        
t_image = np.array(t_image)
t_lbp=np.array(t_lbp)
t_image_glcm=np.array(t_image_glcm)
t_hist=np.array(t_hist)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(trait_labels_lbp)
train_labels_encoded = le.transform(trait_labels_lbp)

## Class Balance for LBP

In [ ]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_res_lbp,Y_res_lbp=over.fit_resample(train_images_lbp,train_labels_encoded)

In [ ]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_result_lbp,Y_result_lbp=over.fit_resample(X_res_lbp,Y_res_lbp)

#COLOR HIST

color_hist train

In [ ]:
def hist_features(img):
  hist_blue = cv2.calcHist([img],[0],None,[256],[0,256]) #blue
  hist_green = cv2.calcHist([img],[1],None,[256],[0,256])
  hist_red = cv2.calcHist([img],[2],None,[256],[0,256])
  hist=np.hstack([hist_green,hist_red,hist_blue])
  return hist.flatten()

In [ ]:
#Resize images to
SIZE = 224

#Capture images and labels into arrays.
#Start by creating empty lists.
train_hist = []
train_label_hist = [] 
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_train/*"):
    labelhist = directory_path.split("\\")[-1]
    label = directory_path.split("/")[-1]
    #print(labelhist)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img3 = cv2.imread(img_path,cv2.COLOR_BGR2RGB) #Reading color images
        img3 = cv2.resize(img3, (SIZE, SIZE)) #Resize images
        img3=hist_features(img3) #call the function
        train_hist.append(img3)
        train_label_hist.append(labelhist)

train_hist = np.array(train_hist)
train_label_hist= np.array(train_label_hist)

color hist test

In [ ]:
test_hist = []
#test_image_ID= []
#hu_features_test = []
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX/*")):
    test_label = directory_path.split("/")[-1]
    print(test_label)
    #test_image_ID.append(test_label)
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
        #print(img_path)
        img4 = cv2.imread(img_path,cv2.COLOR_BGR2RGB) #Reading color images
        img4 = cv2.resize(img4, (SIZE, SIZE)) #Resize images
        img4=hist_features(img4)
        test_hist.append(img4)

test_hist = np.array(test_hist)

In [ ]:
le.fit(train_label_hist)
train_labels_encoded_hist = le.transform(train_label_hist)

In [ ]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_res_colorHist,Y_res_colorHist=over.fit_resample(train_hist,train_labels_encoded_hist)

In [ ]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_result_colorHist,Y_result_colorHist=over.fit_resample(X_res_colorHist,Y_res_colorHist)

#GLCM

glcm train data

In [ ]:
def feature_extractor(dataset):
  image_dataset  = pd.DataFrame()
  distance = [11, 21]
  angle = [0, np.pi/4, np.pi/2, 3*np.pi/4]
  for image in range(dataset.shape[0]):
    img = dataset[image]
    df = pd.DataFrame()
    count = 1
    for dist in distance:
      for ang in angle:
        GLCM = greycomatrix(img, [dist], [ang])      
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df[f'Energy{count}'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df[f'Corr{count}'] = GLCM_corr             
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df[f'Homogen{count}'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df[f'Contrast{count}'] = GLCM_contr
        count = count + 1
    image_dataset = image_dataset.append(df)    
  return image_dataset

In [ ]:
#Resize images to
SIZE = 224

#Capture images and labels into arrays.
#Start by creating empty lists.
train_image_glcm = []
train_label_glcm = [] 
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_train/*"):
    label_glcm = directory_path.split("\\")[-1]
    label = directory_path.split("/")[-1]
    #print(label_glcm)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        img5= cv2.imread(img_path,0)
        #img5 = cv2.imread(img_path,cv2.COLOR_BGR2RGB) #Reading color images
        img5 = cv2.resize(img5, (SIZE, SIZE)) #Resize images
        train_image_glcm.append(img5)
        train_label_glcm.append(label_glcm)


train_label_glcm= np.array(train_label_glcm) 
train_image_glcm = np.array(train_image_glcm)

In [ ]:
train_image_glcm.shape

(5316, 224, 224)

In [ ]:
#x_train=train_image_glcm

In [ ]:
train_feature_glcm = feature_extractor(train_image_glcm)

In [ ]:
train_feature_glcm.shape

(5316, 32)

test data for glcm

In [ ]:
test_image_glcm = []
#test_image_ID= []
#hu_features_test = []
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX/*")):
    #test_label = directory_path.split("/")[-1]
    print(test_label)
    #test_image_ID.append(test_label)
for directory_path in sorted(glob.glob("/content/drive/MyDrive/testX")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
        #print(img_path)
        img6= cv2.imread(img_path,0)
        img6 = cv2.resize(img6, (SIZE, SIZE))
        test_image_glcm.append(img6)
  
test_image_glcm = np.array(test_image_glcm)

In [ ]:
le.fit(train_label_glcm)
train_labels_encoded_glcm = le.transform(train_label_glcm)

In [ ]:
test_feature_glcm = feature_extractor(test_image_glcm)

concat lbp and color 


In [ ]:
train_images_lbp.shape

(5316, 26)

In [ ]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_res_glcm,Y_res_glcm=over.fit_resample(train_feature_glcm,train_labels_encoded_glcm)

In [ ]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_result_glcm,Y_result_glcm=over.fit_resample(X_res_glcm,Y_res_glcm)

#Feature Concatination (LBP + Color Histogram + GLCM)

In [ ]:
train_feature_final=np.concatenate((X_result_lbp,X_result_colorHist,X_result_glcm),axis=1)
test_feature_final=np.concatenate((test_images_lbp,test_hist,test_feature_glcm),axis=1)


In [ ]:
test_images_all = np.concatenate((test_images_lbp,test_hist,test_feature_glcm),axis=1)

#Random forest

In [ ]:
pipe = Pipeline([
                 ("scale", StandardScaler()),
                 ("classifier", RandomForestClassifier(n_estimators = 800, max_depth=60,random_state = 42))
                 ])

In [ ]:
pipe.fit(train_feature_final,Y_result_glcm)

Pipeline(steps=[('scale', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(max_depth=60, n_estimators=800,
                                        random_state=42))])

In [ ]:
prediction = pipe.predict(test_images_all)